In [1]:
import numpy as np
import cobaya

#fiducial_params = {
#    "ombh2": 0.02225,
#    "omch2": 0.1198,
#    "H0": 67.3,
#    "tau": 0.06,
#    "As": 2.2e-9,
#    "ns": 0.96,
#    "mnu": 0.06,
#    "nnu": 3.046,
#    "num_massive_neutrinos": 3,
#    }

fiducial_params = {'omch2': 0.125,     #FOR Testing against NEMO
                    'ombh2': 0.02225,
                    'H0': 70.,
                    'ns': 0.95,
                    'As': 1.88e-9,
                    'mnu': 0.0,
                    'tau': 0.06,
                    'nnu': 3.046,
                    'num_massive_neutrinos':0}

info_fiducial = {
    "params": fiducial_params,
    "likelihood": {"soliket.ClusterLikelihood": {"stop_at_error": True,}},
    "theory": {
        "camb": {
            "extra_args": {
                "accurate_massive_neutrino_transfers": True,
                "redshifts": np.linspace(0, 2, 41),
                "nonlinear": False,
                "kmax": 10.0,
                "dark_energy_model": "ppf",
            }
        }
    },
    'modules': '/Users/nab/Repos/cobaya_modules/'
}

from cobaya.model import get_model

model_fiducial = get_model(info_fiducial)

lnl = model_fiducial.loglikes({})[0]

#assert np.isfinite(lnl)

print (lnl)

[prior] *WARNING* No sampled parameters requested! This will fail for non-mock samplers.
[camb] Importing *local* CAMB from /Users/nab/Repos/cobaya_modules/code/CAMB
mock catalog
Ntot 198.764436582616
Pfunc per 3127.328648620607 0.7560644
Pfunc per 308.39815016323877 0.9348018
Pfunc per 4916.08740531354 0.49927437
Pfunc per 1584.9611608461375 0.35925263
Pfunc per 4958.828772152223 0.44209617
Pfunc per 5200.934356260367 0.16338149
Pfunc per 1844.3969329306387 0.924398
Pfunc per 5115.940614132112 0.45040905
Pfunc per 4669.053393471624 0.44943744
Pfunc per 3461.6792713867485 0.18964909
Pfunc per 3316.478391728262 0.47681066
Pfunc per 3446.5155460808214 0.2859734
Pfunc per 1215.2785944237069 0.8776922
Pfunc per 3072.673833880557 0.62664866
Pfunc per 5875.639991584301 0.21387655
Pfunc per 241.52559912701585 0.53458494
Pfunc per 2869.015128602843 0.3563333
Pfunc per 343.8792789767118 0.77571094
Pfunc per 2408.504009317666 0.21330465
Pfunc per 194.83192364723277 0.42216015
Pfunc per 4885.7091

In [ ]:
3.20111782e+03 

In [ ]:
import numpy as np
def to_array(s):
    return np.array(s[1:-1].split(),dtype=float)

In [ ]:
cob = to_array('''[1.91779243e+03 2.02043522e+03 2.12855762e+03 2.24244963e+03
 2.36241634e+03 2.48877875e+03 2.62187453e+03 2.76205814e+03
 2.90970115e+03 3.06519399e+03 3.22894690e+03 3.40139093e+03
 3.58297892e+03 3.77418527e+03 3.97550527e+03 4.18745804e+03
 4.41058787e+03 4.64546533e+03 4.89268844e+03 5.15288194e+03
 5.42669391e+03 5.71480064e+03 6.01790860e+03 6.33675557e+03
 6.67211169e+03 7.02477837e+03 7.39558064e+03 7.78537346e+03
 8.19504490e+03 8.62551693e+03 9.07774596e+03 9.55272067e+03
 1.00514449e+04 1.05749451e+04 1.11242775e+04 1.17005267e+04
 1.23048059e+04 1.29382521e+04 1.36019888e+04 1.42971316e+04
 1.50248051e+04 1.57861398e+04 1.65822683e+04 1.74143183e+04
 1.82833544e+04 1.91903699e+04 2.01363175e+04 2.11221033e+04
 2.21485784e+04 2.32165276e+04 2.43266008e+04 2.54792770e+04
 2.66749101e+04 2.79137192e+04 2.91957749e+04 3.05209844e+04
 3.18890065e+04 3.32991725e+04 3.47505614e+04 3.62419926e+04
 3.77720101e+04 3.93388662e+04 4.09403760e+04 4.25738469e+04
 4.42362560e+04 4.59240163e+04 4.76331064e+04 4.93588929e+04
 5.10982056e+04 5.28417581e+04 5.45838221e+04 5.63176167e+04
 5.80352782e+04 5.97262636e+04 6.13841174e+04 6.29975068e+04
 6.45553581e+04 6.60468956e+04 6.74577059e+04 6.87752551e+04
 6.99847667e+04 7.10720822e+04 7.20214786e+04 7.28172055e+04
 7.34432459e+04 7.38835866e+04 7.41227154e+04 7.41457834e+04
 7.39392103e+04 7.34913294e+04 7.27928765e+04 7.18378044e+04
 7.06240224e+04 6.91542061e+04 6.74366392e+04 6.54858826e+04
 6.33234412e+04 6.09780869e+04 5.84858334e+04 5.58895332e+04
 5.32378205e+04 5.05840145e+04 4.79630796e+04 4.54224823e+04
 4.30385157e+04 4.08557009e+04 3.88968418e+04 3.71592962e+04
 3.56181247e+04 3.42350631e+04 3.29663004e+04 3.18098809e+04
 3.05843590e+04 2.92501650e+04 2.77064598e+04 2.59132781e+04
 2.39074933e+04 2.17954123e+04 1.97133494e+04 1.77946499e+04
 1.61557214e+04 1.48721871e+04 1.39382623e+04 1.32485331e+04
 1.26198168e+04 1.18527091e+04 1.08509723e+04 9.69437476e+03
 8.55992287e+03 7.63650650e+03 7.01472218e+03 6.58542901e+03
 6.15596301e+03 5.58909113e+03 4.93536714e+03 4.36863082e+03
 3.98167143e+03 3.68399492e+03 3.34031328e+03 2.95568751e+03
 2.63780905e+03 2.40219255e+03 2.16883372e+03 1.92501648e+03
 1.72420458e+03 1.55354656e+03 1.38366898e+03 1.23456422e+03
 1.10486489e+03 9.82938251e+02 8.76280481e+02 7.79834652e+02
 6.93233007e+02 6.16067978e+02 5.46779357e+02 4.85137772e+02
 4.30017945e+02 3.80822296e+02 3.36995234e+02 2.98014280e+02
 2.63374515e+02 2.32611045e+02 2.05306795e+02 1.81088539e+02
 1.59624748e+02 1.40617952e+02 1.23800464e+02 1.08931516e+02
 9.57944513e+01 8.41952818e+01 7.39606670e+01 6.49358828e+01
 5.69829716e+01 4.99789879e+01 4.38144578e+01 3.83919637e+01
 3.36248822e+01 2.94363101e+01 2.57580338e+01 2.25295874e+01
 1.96974140e+01 1.72141215e+01 1.50378042e+01 1.31314262e+01
 1.14622810e+01 1.00015115e+01 8.72367363e+00 7.60634294e+00
 6.62977051e+00 5.77657697e+00 5.03147658e+00 4.38103006e+00
 3.81343009e+00 3.31831045e+00 2.88657462e+00 2.51024377e+00
 2.18232329e+00 1.89668454e+00 1.64795992e+00 1.43144986e+00]''')

szr = to_array('''[3.99959066e+02 4.21378684e+02 4.43944152e+02 4.67716481e+02
 4.92759892e+02 5.19142053e+02 5.46934224e+02 5.76211346e+02
 6.07052230e+02 6.39539788e+02 6.73761212e+02 7.09808222e+02
 7.47777240e+02 7.87769667e+02 8.29892034e+02 8.74256282e+02
 9.20979975e+02 9.70186506e+02 1.02200542e+03 1.07657251e+03
 1.13403026e+03 1.19452802e+03 1.25822238e+03 1.32527715e+03
 1.39586363e+03 1.47016101e+03 1.54835636e+03 1.63064521e+03
 1.71723131e+03 1.80832725e+03 1.90415433e+03 2.00494291e+03
 2.11093202e+03 2.22236882e+03 2.33950982e+03 2.46262043e+03
 2.59197553e+03 2.72785797e+03 2.87055917e+03 3.02037819e+03
 3.17762108e+03 3.34260128e+03 3.51563765e+03 3.69705536e+03
 3.88718255e+03 4.08635126e+03 4.29489467e+03 4.51314609e+03
 4.74143838e+03 4.98010073e+03 5.22945874e+03 5.48982857e+03
 5.76151810e+03 6.04482211e+03 6.34002097e+03 6.64737481e+03
 6.96711777e+03 7.29945766e+03 7.64456484e+03 8.00257470e+03
 8.37357443e+03 8.75760361e+03 9.15464354e+03 9.56461105e+03
 9.98735657e+03 1.04226508e+04 1.08701852e+04 1.13295465e+04
 1.18002235e+04 1.22815761e+04 1.27728351e+04 1.32731059e+04
 1.37813561e+04 1.42964163e+04 1.48169507e+04 1.53414691e+04
 1.58682528e+04 1.63953844e+04 1.69207645e+04 1.74421085e+04
 1.79569088e+04 1.84623275e+04 1.89553252e+04 1.94326115e+04
 1.98907127e+04 2.03258626e+04 2.07340482e+04 2.11110486e+04
 2.14523661e+04 2.17534120e+04 2.20093514e+04 2.22153670e+04
 2.23665450e+04 2.24580787e+04 2.24853736e+04 2.24441070e+04
 2.23305342e+04 2.21415541e+04 2.18749713e+04 2.15298022e+04
 2.11063629e+04 2.06067485e+04 2.00348197e+04 1.93965339e+04
 1.87000904e+04 1.79557425e+04 1.71757830e+04 1.63744588e+04
 1.55671968e+04 1.47699834e+04 1.39986753e+04 1.32679402e+04
 1.25901552e+04 1.19742686e+04 1.14246773e+04 1.09401262e+04
 1.05129124e+04 1.01284973e+04 9.76586715e+03 9.39921540e+03
 9.00121925e+03 8.54819731e+03 8.02624443e+03 7.43761673e+03
 6.80300629e+03 6.15983533e+03 5.55452066e+03 5.02967760e+03
 4.61256052e+03 4.30514229e+03 4.08076831e+03 3.88899397e+03
 3.67147400e+03 3.38891165e+03 3.04581748e+03 2.69135197e+03
 2.38890399e+03 2.17480928e+03 2.03390299e+03 1.91027950e+03
 1.75082800e+03 1.55436243e+03 1.36977916e+03 1.23744874e+03
 1.14442938e+03 1.04645567e+03 9.29988839e+02 8.25070490e+02
 7.48608344e+02 6.79602744e+02 6.04786010e+02 5.39255810e+02
 4.86364586e+02 4.34525924e+02 3.86822224e+02 3.46462947e+02
 3.08584183e+02 2.74839080e+02 2.44870862e+02 2.17619114e+02
 1.93491423e+02 1.71746051e+02 1.52405141e+02 1.35114082e+02
 1.19686669e+02 1.05941447e+02 9.37100091e+01 8.28346085e+01
 7.31720589e+01 6.45935622e+01 5.69836560e+01 5.02384125e+01
 4.42642726e+01 3.89770959e+01 3.43013447e+01 3.01692810e+01
 2.65202037e+01 2.32998216e+01 2.04596366e+01 1.79563649e+01
 1.57514191e+01 1.38104293e+01 1.21028025e+01 1.06013436e+01
 9.28190080e+00 8.12303818e+00 7.10575194e+00 6.21320868e+00
 5.43050691e+00 4.74446732e+00 4.14344229e+00 3.61714382e+00
 3.15649242e+00 2.75348061e+00 2.40105002e+00 2.09298374e+00
 1.82380905e+00 1.58870882e+00 1.38344832e+00 1.20430917e+00]''')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
k = np.logspace(-4, np.log10(5), 200)
plt.loglog(k/0.673,cob*0.673**3,label='cobaya')
plt.plot(k,szr,label='szar')
plt.legend()

plt.figure()
plt.semilogx(k,cob*0.673**3/szr)